In [22]:
import argparse
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import optparse
from dotenv import load_dotenv
import pickle

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets
from brainflow import ml_model
from brainflow.ml_model import *


In [33]:
def get_data():
    load_dotenv()

    BoardShim.enable_dev_board_logger()

    board_id = 0
    serial_port = os.getenv("SERIAL_PORT")

    params = BrainFlowInputParams()
    params.serial_port = serial_port
    board = BoardShim(board_id, params)
    board.prepare_session()
    sampling_rate = board.get_sampling_rate(board_id)   # per second, presumably

    board.start_stream()
    t = 11
    time.sleep(t)
    # data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer
    data = board.get_board_data()  # get all data and remove it from internal buffer
    board.stop_stream()
    board.release_session()

    return sampling_rate, data

In [35]:
data = get_data()

BrainFlowError: BOARD_NOT_READY_ERROR:7 unable to prepare streaming session

In [20]:
d2 = data[1:8, :2406]

In [23]:
dfft =  np.fft.fft(d2).view(np.float64).reshape(7, 2406, 2)

In [24]:
dfft.shape

(7, 2406, 2)

In [29]:
model = pickle.load(open("finalized_model.sav", 'rb'))

In [32]:
model.predict(dfft.reshape(1, -1))

array([1.])